<a href="https://colab.research.google.com/github/jfogarty/machine-learning-intro-workshop/blob/master/misc/pyspark_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PySpark in Google Colab

- From [PySpark in Google Colab](https://towardsdatascience.com/pyspark-in-google-colab-6821c2faf41c) in [towardsdatascience.com](https://towardsdatascience.com) by [Asif Ahmed](https://github.com/asifahmed90)
Creating a simple linear regression model with PySpark in Colab

Updated by [John Fogarty](https://github.com/jfogarty) for Python 3.6 and [Base2 MLI](https://github.com/base2solutions/mli) and [colab](https://colab.research.google.com) standalone evaluation.

With broadening sources of the data pool, the topic of Big Data has received an increasing amount of attention in the past few years. Besides dealing with the gigantic data of all kinds and shapes, the target turnaround time of the analysis part for the big data has been reduced significantly. Not only has this speed and efficiency helped in the immediate analysis of the Big Data but also in identifying new opportunities. This, in turn, has lead to smarter business moves, more efficient operations, higher profits, and happier customers.

Apache Spark was build to analyze Big Data with faster speed. One of the important features that Apache Spark offers is the ability to run the computations in memory. It is also considered to be more efficient than MapReduce for the complex application running on Disk.

Spark is designed to be highly accessible, offering simple APIs in Python, Java, Scala, and SQL, and rich built-in libraries. It also integrates closely with other Big Data tools. In particular, Spark can run in Hadoop clusters and access any Hadoop data source, including Cassandra.

PySpark is the interface that gives access to Spark using the Python programming language. PySpark is an API developed in python for spark programming and writing spark applications in Python style, although the underlying execution model is the same for all the API languages.
In this tutorial, we will mostly deal with the PySpark machine learning library Mllib that can be used to import the Linear Regression model or other machine learning models.

### Yes, but why Google Colab?

Colab by Google is based on Jupyter Notebook which is an incredibly powerful tool that leverages google docs features. Since it runs on google server, we don't need to install anything in our system locally, be it Spark or deep learning model. The most attractive features of Colab are the free GPU and TPU support! Since the GPU support runs on Google's own server, it is, in fact, faster than some commercially available GPUs like the Nvidia 1050Ti. A piece of general system information allocated for a user looks like the following:

```
Gen RAM Free: 11.6 GB  | Proc size: 666.0 MB
GPU RAM Free: 11439MB | Used: 0MB | Util  0% | Total 11439MB
```


## Running Pyspark in Colab

To run spark in Colab, first we need to install all the dependencies in Colab environment such as Apache Spark 2.3.2 with hadoop 2.7, Java 8 and Findspark in order to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab.

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.3/spark-2.4.3-bin-hadoop2.7.tgz
!tar xf spark-2.4.3-bin-hadoop2.7.tgz
!pip install -q findspark

**Note!** This was out of date and had to be updated from [apache spark](https://www-us.apache.org/dist/spark) to 2.4.3 form 2.4.1 before it would install.

Now that we have installed Spark and Java in Colab, it is time to set the environment path that enables us to run PySpark in our Colab environment. Set the location of Java and Spark by running the following code:


In [0]:
import os
JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"
SPARK_HOME = "/content/spark-2.4.3-bin-hadoop2.7"

def set_os_environ_path(var, val):
    os.environ[var] = val
    if not os.path.exists(JAVA_HOME):
        print(f"** Yikes! the {var} path {val} does not exist!  Your environment is not valid.")

set_os_environ_path("JAVA_HOME",  JAVA_HOME)
set_os_environ_path("SPARK_HOME", SPARK_HOME)


**Note!** You **must** check these paths in the **Files** tab on the left side of your notebook page.  

We can run a local spark session to test our installation:

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

## Linear Regression Model

Linear Regression model is one the oldest and widely used machine learning approach which assumes a relationship between dependent and independent variables. For example, a modeler might want to predict the forecast of the rain based on the humidity ratio. Linear Regression consists of the best fitting line through the scattered points on the graph and the best fitting line is known as the regression line. Detailed about linear regression can be found here.

For our purpose of starting with Pyspark in Colab and to keep things simple, we will use the famous Boston Housing dataset. A full description of this dataset can be found in this [link](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html).

### The Boston Housing Dataset

A Dataset derived from information collected by the U.S. Census Service concerning housing in the area of Boston Mass.
BackUpDelve

This dataset contains information collected by the U.S Census Service concerning housing in the area of Boston Mass. It was obtained from the [StatLib archive](http://lib.stat.cmu.edu/datasets/boston), and has been used extensively throughout the literature to benchmark algorithms. However, these comparisons were primarily done outside of Delve and are thus somewhat suspect. The dataset is small in size with only 506 cases.

The data was originally published by Harrison, D. and Rubinfeld, D.L. `Hedonic prices and the demand for clean air', J. Environ. Economics & Management, vol.5, 81-102, 1978.

#### Dataset Naming
The name for this dataset is simply boston. It has two prototasks: nox, in which the nitrous oxide level is to be predicted; and price, in which the median value of a home is to be predicted

#### Miscellaneous Details

- Origin : The origin of the boston housing data is Natural.
- Usage : This dataset may be used for Assessment.
- Number of Cases : The dataset contains a total of 506 cases.
- Order : The order of the cases is mysterious.
- Variables : There are 14 attributes in each case of the dataset. They are:
  - CRIM - per capita crime rate by town
  - ZN - proportion of residential land zoned for lots over 25,000 sq.ft.
  - INDUS - proportion of non-retail business acres per town.
  - CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)
  - NOX - nitric oxides concentration (parts per 10 million)
  - RM - average number of rooms per dwelling
  - AGE - proportion of owner-occupied units built prior to 1940
  - DIS - weighted distances to five Boston employment centres
  - RAD - index of accessibility to radial highways
  - TAX - full-value property-tax rate per 10,000 dollars
  - PTRATIO - pupil-teacher ratio by town
  - B - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
  - LSTAT - % lower status of the population
  - MEDV - Median value of owner-occupied homes in $1000's

- Note : Variable #14 seems to be censored at 50.00 \(corresponding to a median price of 50,000 dollars);

 Censoring is suggested by the fact that the highest median price of exactly 50,000 dollars is reported in 16 cases, while 15 cases have prices between 40,000 dollars  and 50,000 dollars , with prices rounded to the nearest hundred. Harrison and Rubinfeld do not mention any censoring.

### Getting the dataset

The goal of this exercise is to predict the housing prices from the given features. Let’s predict the prices of the Boston Housing dataset by considering MEDV as the target variable and all other variables as input features.

We can download the dataset from this [Github repo: We can download the dataset from this [Github repo: link](https://github.com/asifahmed90/pyspark-ML-in-Colab](https://github.com/asifahmed90/pyspark-ML-in-Colab/blob/master/BostonHousing.csv) and keep it somewhere accessible in our local drives. The dataset can be loaded in the Colab directory using the following command from the same drive.

```
    from google.colab import files
    files.upload()
```

**JF Note : this is tedious so instead let's fetch it directly from the raw Github content using this code:

In [4]:
Github_REPO = 'https://github.com/asifahmed90/pyspark-ML-in-Colab/'
REPO        = 'https://raw.githubusercontent.com/asifahmed90/pyspark-ML-in-Colab/'
BRANCH      = 'master/'
filename    = 'BostonHousing.csv'
toDir       = 'tmpData'
toPath      =  os.path.join(toDir, filename)

URL = os.path.join(REPO, BRANCH, filename)
if not os.path.exists(toDir): os.makedirs(toDir)
!wget $URL -O $toPath

--2019-08-16 22:33:09--  https://raw.githubusercontent.com/asifahmed90/pyspark-ML-in-Colab/master/BostonHousing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35735 (35K) [text/plain]
Saving to: ‘tmpData/BostonHousing.csv’

tmpData/BostonHousi 100%[===================>]  34.90K  --.-KB/s    in 0.01s   

2019-08-16 22:33:09 (2.75 MB/s) - ‘tmpData/BostonHousing.csv’ saved [35735/35735]



We can now check the directory content of the Colab `/content` directory.  Note that your colab is running in a full VM instance and you are installing new packages into the root wil full superuser privileges.

In [15]:
!pwd ; ls -al tmpData

/content
total 44
drwxr-xr-x 2 root root  4096 Aug 16 22:33 .
drwxr-xr-x 1 root root  4096 Aug 16 22:33 ..
-rw-r--r-- 1 root root 35735 Aug 16 22:33 BostonHousing.csv


We should see a file named BostonHousing.csv saved. Now that we have uploaded the dataset successfully, we can start analyzing.

For our [linear regression](https://en.wikipedia.org/wiki/Linear_regression) model, we need to import [Vector Assembler](https://spark.apache.org/docs/2.2.0/ml-features.html) and [Linear Regression](https://spark.apache.org/docs/2.1.1/ml-classification-regression.html) modules from the [PySpark API](). Vector Assembler is a transformer tool that assembles all the features into one vector from multiple columns that contain type [double](https://en.wikipedia.org/wiki/Double-precision_floating-point_format). We should have used (must use) [StringIndexer](https://spark.rstudio.com/reference/ft_string_indexer/) if any of our columns contains string values to convert it into numeric values. Luckily, the BostonHousing dataset only contains type double, so we can skip StringIndexer for now.

In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
dataset = spark.read.csv(toPath, inferSchema=True, header=True)

Notice that we used InferSchema inside read.csv(). InferSchema automatically infers different data types for each column.
Let us print look into the dataset to see the data types of each column:

In [7]:
dataset.printSchema()

root
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: integer (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: integer (nullable = true)
 |-- tax: integer (nullable = true)
 |-- ptratio: double (nullable = true)
 |-- b: double (nullable = true)
 |-- lstat: double (nullable = true)
 |-- medv: double (nullable = true)



In [14]:
dataset.head(10)

[Row(crim=0.00632, zn=18.0, indus=2.31, chas=0, nox=0.538, rm=6.575, age=65.2, dis=4.09, rad=1, tax=296, ptratio=15.3, b=396.9, lstat=4.98, medv=24.0),
 Row(crim=0.02731, zn=0.0, indus=7.07, chas=0, nox=0.469, rm=6.421, age=78.9, dis=4.9671, rad=2, tax=242, ptratio=17.8, b=396.9, lstat=9.14, medv=21.6),
 Row(crim=0.02729, zn=0.0, indus=7.07, chas=0, nox=0.469, rm=7.185, age=61.1, dis=4.9671, rad=2, tax=242, ptratio=17.8, b=392.83, lstat=4.03, medv=34.7),
 Row(crim=0.03237, zn=0.0, indus=2.18, chas=0, nox=0.458, rm=6.998, age=45.8, dis=6.0622, rad=3, tax=222, ptratio=18.7, b=394.63, lstat=2.94, medv=33.4),
 Row(crim=0.06905, zn=0.0, indus=2.18, chas=0, nox=0.458, rm=7.147, age=54.2, dis=6.0622, rad=3, tax=222, ptratio=18.7, b=396.9, lstat=5.33, medv=36.2),
 Row(crim=0.02985, zn=0.0, indus=2.18, chas=0, nox=0.458, rm=6.43, age=58.7, dis=6.0622, rad=3, tax=222, ptratio=18.7, b=394.12, lstat=5.21, medv=28.7),
 Row(crim=0.08829, zn=12.5, indus=7.87, chas=0, nox=0.524, rm=6.012, age=66.6, di

In the next step, we will convert all the features from different columns into a single column and we can call the new vector column as ‘Attributes’ in the outputCol.

In [8]:
#Input all the features in one vector column
assembler = VectorAssembler(inputCols=['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat'], outputCol = 'Attributes')
output = assembler.transform(dataset)
#Input vs Output
finalized_data = output.select("Attributes","medv")
finalized_data.show()

+--------------------+----+
|          Attributes|medv|
+--------------------+----+
|[0.00632,18.0,2.3...|24.0|
|[0.02731,0.0,7.07...|21.6|
|[0.02729,0.0,7.07...|34.7|
|[0.03237,0.0,2.18...|33.4|
|[0.06905,0.0,2.18...|36.2|
|[0.02985,0.0,2.18...|28.7|
|[0.08829,12.5,7.8...|22.9|
|[0.14455,12.5,7.8...|27.1|
|[0.21124,12.5,7.8...|16.5|
|[0.17004,12.5,7.8...|18.9|
|[0.22489,12.5,7.8...|15.0|
|[0.11747,12.5,7.8...|18.9|
|[0.09378,12.5,7.8...|21.7|
|[0.62976,0.0,8.14...|20.4|
|[0.63796,0.0,8.14...|18.2|
|[0.62739,0.0,8.14...|19.9|
|[1.05393,0.0,8.14...|23.1|
|[0.7842,0.0,8.14,...|17.5|
|[0.80271,0.0,8.14...|20.2|
|[0.7258,0.0,8.14,...|18.2|
+--------------------+----+
only showing top 20 rows



Here, ‘Attributes’ are the input features from all the columns and ‘medv’ is the target column.
Next, we should split the training and testing data according to our dataset (0.8 and 0.2 in this case).

The predicted score in the prediction column is output:

In [9]:
#Split training and testing data
train_data,test_data = finalized_data.randomSplit([0.8,0.2])
regressor = LinearRegression(featuresCol = 'Attributes', labelCol = 'medv')
#Learn to fit the model from training set
regressor = regressor.fit(train_data)
#To predict the prices on testing set
pred = regressor.evaluate(test_data)
#Predict the model
pred.predictions.show()

+--------------------+----+------------------+
|          Attributes|medv|        prediction|
+--------------------+----+------------------+
|[0.01381,80.0,0.4...|50.0| 40.65635174767654|
|[0.01432,100.0,1....|31.6| 32.79076205626436|
|[0.0187,85.0,4.15...|23.1|25.328092365951157|
|[0.03049,55.0,3.7...|31.2| 28.34828025471483|
|[0.03306,0.0,5.19...|20.6|21.677417812351322|
|[0.03466,35.0,6.0...|19.4| 23.35429611036832|
|[0.0351,95.0,2.68...|48.5| 41.57087672180137|
|[0.03659,25.0,4.8...|24.8|25.665461132339455|
|[0.03768,80.0,1.5...|34.6| 35.20955329640833|
|[0.04544,0.0,3.24...|19.8|21.944004738122157|
|[0.0459,52.5,5.32...|22.3|27.166540621750112|
|[0.04819,80.0,3.6...|21.9| 24.22041590295956|
|[0.05023,35.0,6.0...|17.1|20.167919588983963|
|[0.05425,0.0,4.05...|24.6| 28.51046731172827|
|[0.05735,0.0,4.49...|26.6|27.780641160789287|
|[0.06642,0.0,4.05...|29.9| 31.15680679168905|
|[0.06899,0.0,25.6...|22.0| 21.58651972592626|
|[0.06911,45.0,3.4...|30.5|30.223410211427485|
|[0.07022,0.0

We can also print the coefficient and intercept of the regression model by using the following command:

In [10]:
#coefficient of the regression model
coeff = regressor.coefficients
#X and Y intercept
intr = regressor.intercept
print ("The coefficient of the model is : %a" %coeff)
print ("The Intercept of the model is : %f" %intr)

The coefficient of the model is : DenseVector([-0.1056, 0.035, 0.0322, 3.0522, -18.8513, 4.5398, 0.0064, -1.1837, 0.2373, -0.0109, -0.9549, 0.0081, -0.4347])
The Intercept of the model is : 30.198533


Once we are done with the basic linear regression operation, we can go a bit further and analyze our model statistically by importing [RegressionEvaluator](https://jaceklaskowski.gitbooks.io/mastering-apache-spark/spark-mllib/spark-mllib-RegressionEvaluator.html) module from Pyspark.

In [11]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol="medv", predictionCol="prediction", metricName="rmse")
# Root Mean Square Error
rmse = eval.evaluate(pred.predictions)
print("- Root Mean Square Error        RMSE: %6.3f" % rmse)
# Mean Square Error
mse = eval.evaluate(pred.predictions, {eval.metricName: "mse"})
print("- Mean Square Error              MSE: %6.3f" % mse)
# Mean Absolute Error
mae = eval.evaluate(pred.predictions, {eval.metricName: "mae"})
print("- Mean Absolute Error            MAE: %6.3f" % mae)
# r2 - coefficient of determination
r2 = eval.evaluate(pred.predictions, {eval.metricName: "r2"})
print("- Coefficient of determination    r2: %6.3f" %r2)

- Root Mean Square Error        RMSE:  5.640
- Mean Square Error              MSE: 31.814
- Mean Absolute Error            MAE:  3.421
- Coefficient of determination    r2:  0.670


That’s it. You have created your first machine learning model using Pyspark in Google Colab.

You can access the full code from in github from [here](https://github.com/asifahmed90/pyspark-ML-in-Colab/blob/master/PySpark_Regression_Analysis.ipynb).

Please let [me](https://github.com/asifahmed90) know if you run into any other newbie problems that I might be able to help you with. I’d love to help you if I can!

### End of notebook.